In [1]:
input_file = "input_files/day_20.txt"

with open(input_file) as lines:
    data = lines.read().splitlines()

## The Modules

Sending a pulse to a module by calling the instance.

In [2]:
from collections import deque, namedtuple


Event = namedtuple('Event', ['to', 'pulse', 'sender', 'push_count'])


class Module:
    def __init__(self, name):
        self.name = name
        self.state = False
        self.listeners = []
        self.calls = {}

    def broadcast(self, pulse, caller, events, push_count):
        for listener in self.listeners:
            events.append((listener, pulse, self, push_count))

    def __repr__(self):
        return f"{self.__class__.__name__}({self.name})"


class FlipFlop(Module):
    '''
    If a flip-flop module receives a high pulse, it is ignored and nothing happens. 
    However, if a flip-flop module receives a low pulse, it flips between on and off. 
    '''    
    def __call__(self, pulse, caller, events, push_count):
        if pulse:
            return
        self.state = not self.state
        pulse = self.state
        self.broadcast(pulse, self, events, push_count)

        
class Conjunction(Module):
    '''
    If a flip-flop module receives a high pulse, it is ignored and nothing happens. 
    However, if a flip-flop module receives a low pulse, it flips between on and off. 
    '''    
    def __call__(self, pulse, caller, events, push_count):
        '''
         When a pulse is received, the conjunction module first updates its memory for that input. 
         Then, if it remembers high pulses for all inputs, it sends a low pulse; 
         otherwise, it sends a high pulse.
        '''
        self.calls[caller.name] = pulse
        pulse = not all(self.calls.values())
        self.broadcast(pulse, self, events, push_count)
                 
class Broadcast(Module):        
    def __call__(self, pulse, caller, events, push_count):
        self.broadcast(pulse, self, events, push_count)

    
class Output(Module):
    def __call__(self, pulse, caller, events, push_count):
        pass

    
class Button():
    def __init__(self, events, modules):
        self.name = "The Button"
        self.events = events
        self.modules = modules
        self.history = []

    def push(self, i):
        self.history.append(Event(self.modules['broadcaster'], False, self, i))
        self.modules['broadcaster'](False, self, self.events, i)

        while len(self.events):
            f, flag, sender, push_count = self.events.popleft()
            self.history.append(Event(f, flag, sender, push_count))
            f(flag, sender, self.events, i)


## Parse input into modules

In [3]:
def init_modules():
    modules = {}

    for line in data:
        fr, to = line.split(' -> ')
        if fr.startswith('%') or fr.startswith('&'):
            name = fr[1:] 
            m = FlipFlop(name) if fr.startswith('%') else Conjunction(name)
            modules[name] = m
        elif fr == 'broadcaster':
            m = Broadcast(fr)
            modules[fr] = m
        else:
            m = Testing(fr)
            modules[fr] = m

    for line in data:
        fr, to = line.split(' -> ')
        name = fr[1:] if fr[0] in '%&' else fr
        for listener in to.split(', '):
            if listener not in modules: # this happens with the output-only modules
                modules[listener] =  Output(listener)
            listener = modules[listener]
            modules[name].listeners.append(listener)
            listener.calls[name] = False                     

    return modules

## Part One
Push the button 1000 times!

In [4]:
modules = init_modules()            

b = Button(deque(), modules)
for i in range(1000):
    b.push(i)
    
total_pulses = len(b.history)
high_pulses = sum(event.pulse for event in b.history)
high_pulses * (total_pulses - high_pulses)

807069600

# Part Two
The `rx` module is only connected to one Conjunction module and will only recieve a low pulse when the Conjuction recieves all high pulses from it's predecessors in the chain. Figure out what these grandparent modules are:

In [5]:
modules = init_modules()            

print(modules['rx'].calls)
print(modules['hf'].calls)

rx_grand_parents = list(modules['hf'].calls.keys())

rx_grand_parents

{'hf': False}
{'nd': False, 'pc': False, 'vd': False, 'tx': False}


['nd', 'pc', 'vd', 'tx']

Figure out when each grandparent Conjuction fires a High pulse. By ispecting the history, this seems to happen on a cycle. So press the button a bunch, then query the history

In [6]:
modules = init_modules()  
b = Button(deque(), modules)

for i in range(1, 10000):
    b.push(i)

In [7]:
cycle_lengths = []

for gp in rx_grand_parents:
    first_few_event = [(gp, event.push_count) for event in b.history if event.sender.name == gp and event.pulse==True]
    print(first_few_event)
    # get the cycle length
    cycle_lengths.append(first_few_event[0][1])
        

[('nd', 4019), ('nd', 8038)]
[('pc', 3881), ('pc', 7762)]
[('vd', 3767), ('vd', 7534)]
[('tx', 3769), ('tx', 7538)]


In [8]:
import math

math.lcm(*cycle_lengths)

221453937522197